### Sentiment Analysis - Real or Fake Disaster Tweets

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('data_set/train.csv')

In [17]:
# train_df = train_data[['id', 'keyword', 'text', 'target']]

train_df = train_data.iloc[0:, 0:5]
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [18]:
train_df.describe()

,id
count,7613.000000
mean,5441.934848
std,3137.116090
min,1.000000
25%,2734.000000
50%,5408.000000
75%,8146.000000
max,10873.000000


In [19]:
real_disaster = train_df[train_df['target'] == '1']
real_disaster.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [20]:
fake_disaster = train_df[train_df['target'] == '0']
fake_disaster.head()

,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0
17,25,NaN,NaN,Summer is lovely,0
18,26,NaN,NaN,My car is so fast,0
19,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0


#### Pre-Processor Method: Remove noise from the text

In [32]:
import re
from string import punctuation

def remove_noise_from_text(review):
    review = re.sub(r'[`=~!@#$%^&*()_+\[\]{};\\:"|<,./<>?^]', ' ', review)
    words = review.split()
    new_review = str()
    for word in words:
        word = word.lower()
        word = word.strip(punctuation)
        word = word.strip()
        new_review += word + " "
    return new_review[:len(new_review)-1]
    

#### Remove noise from Real Disaster training 

In [33]:
for i in range(len(real_disaster)):
    str_data = str(real_disaster['text'].values[i])
    real_disaster['text'].values[i] = remove_noise_from_text(str_data)
real_disaster.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1


#### Remove noise from all fake disaster tweet

In [34]:
for i in range(len(fake_disaster)):
    str_data = fake_disaster['text'].values[i]
    fake_disaster['text'].values[i] = remove_noise_from_text(str_data)
fake_disaster.head()

,id,keyword,location,text,target
15,23,NaN,NaN,what s up man,0
16,24,NaN,NaN,i love fruits,0
17,25,NaN,NaN,summer is lovely,0
18,26,NaN,NaN,my car is so fast,0
19,28,NaN,NaN,what a goooooooaaaaaal,0


### Preprocess Train Data

In [35]:
for i in range(len(train_df)):
    str_data = train_df['text'].values[i]
    train_df['text'].values[i] = remove_noise_from_text(str_data)
train_df.head()

TypeError: expected string or bytes-like object

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

